<h1>preprocessing<h1>

In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [6]:
# read and filter

path = '../data/scraped.csv'
df = pd.read_csv(path)

# filter out for posts w/ >1 comments
df = df[df['num_comments'] > 1]

# filter out these (uninterpretable cols)
df.drop(columns=['title', 'url', 'upvotes', 'num_comments', 'symmetry_td'], inplace=True)

<h1>splitting<h1>

In [ ]:
ohe = OneHotEncoder()
ss = StandardScaler()

In [7]:
df

,subreddit,upvote_ratio,vote-to-comment,brightness,contrast,saturation,symmetry_lr,light_balance_lr,light_balance_td,hue_diversity,dynamic_range,sharpness,thirds_balance,texture_variance,subject_type,genre,perspective,color_mode,complexity,mean_sentiment
0,itookapicture,0.97,70.730359,105.460391,74.716782,67.446851,0.818354,4.897785,83.222995,3.266612,222.0,221.872674,0.187847,54.580493,a photo of architecture or man-made structures,an architectural photo emphasizing buildings o...,a wide or far distance photo emphasizing scene...,a color photo with natural or vivid tones,"a busy photo with many subjects, objects, or v...",0.552969
1,itookapicture,0.96,38.177349,145.243377,32.376147,56.435357,-0.042687,4.547871,6.295794,2.410789,112.0,62.312338,0.104924,41.979576,a photo of an animal,an abstract photo,a medium distance photo showing the subject wi...,a color photo with natural or vivid tones,a simple photo focusing on just a few clear su...,0.565821
2,itookapicture,0.98,136.945330,42.290752,53.746539,50.268145,0.434616,21.206361,2.337741,3.675420,212.0,117.798888,0.254383,42.436616,a photo of architecture or man-made structures,an architectural photo emphasizing buildings o...,a medium distance photo showing the subject wi...,a color photo with natural or vivid tones,"a busy photo with many subjects, objects, or v...",0.628210
3,itookapicture,0.98,114.424370,74.965327,47.962195,119.170866,0.511676,42.852314,34.549881,2.570944,150.0,108.586432,0.252381,25.307158,a nature photo showing scenery or nature,a landscape photo showing scenery or nature,a wide or far distance photo emphasizing scene...,a color photo with natural or vivid tones,a simple photo focusing on just a few clear su...,0.790322
4,itookapicture,0.93,52.416254,52.142366,41.618183,64.044091,0.067495,17.064209,3.824684,2.959793,110.0,16.660357,0.205488,34.453484,a still life photo featuring arranged objects ...,a night or low-light photo with visible artifi...,a wide or far distance photo emphasizing scene...,a color photo with natural or vivid tones,"a busy photo with many subjects, objects, or v...",0.606220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,postprocessing,1.00,204.625000,150.728680,100.248548,70.188972,0.793237,2.179118,131.606163,4.170008,247.0,1297.680210,0.320337,115.395785,a photo of architecture or man-made structures,an architectural photo emphasizing buildings o...,a wide or far distance photo emphasizing scene...,a color photo with natural or vivid tones,"a busy photo with many subjects, objects, or v...",0.716195
3646,postprocessing,0.99,77.500000,96.818428,55.887787,77.979845,0.472658,21.870396,38.342755,3.284399,186.0,285.580108,0.234524,38.849715,a nature photo showing scenery or nature,a night or low-light photo with visible artifi...,a wide or far distance photo emphasizing scene...,a color photo with natural or vivid tones,a simple photo focusing on just a few clear su...,0.810949
3647,postprocessing,0.99,50.593220,76.187088,69.629202,100.299921,0.691937,23.233265,98.564732,3.754661,210.0,1194.738842,0.164027,62.946488,a photo of architecture or man-made structures,a night or low-light photo with visible artifi...,a wide or far distance photo emphasizing scene...,a color photo with natural or vivid tones,"a busy photo with many subjects, objects, or v...",0.713928
3648,postprocessing,0.99,55.698113,59.705122,56.636228,120.575579,0.322900,20.459038,30.686619,2.962174,191.0,236.705929,0.088946,63.380575,a photo of an animal,a night or low-light photo with visible artifi...,a wide or far distance photo emphasizing scene...,a color photo with natural or vivid tones,a simple photo focusing on just a few clear su...,0.547105
